In [1]:
from keras.layers import Conv2D, UpSampling2D
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from skimage.color import rgb2lab, lab2rgb
from skimage.transform import resize
from skimage.io import imsave
import numpy as np
import tensorflow as tf

In [3]:
path = 'images/'

#Normalize images - divide by 255
train_datagen = ImageDataGenerator(rescale=1. / 255)

#Resize images, if needed
train = train_datagen.flow_from_directory(path, 
                                          target_size=(256, 256), 
                                          batch_size=340, 
                                          class_mode=None)

Found 3 images belonging to 2 classes.


In [4]:
#Convert from RGB to Lab
"""
by iterating on each image, we convert the RGB to Lab. 
Think of LAB image as a grey image in L channel and all color info stored in A and B channels. 
The input to the network will be the L channel, so we assign L channel to X vector. 
And assign A and B to Y.
"""

X =[]
Y =[]
for img in train[0]:
    try:
        lab = rgb2lab(img)
        X.append(lab[:,:,0]) 
        Y.append(lab[:,:,1:] / 128) #A and B values range from -127 to 128, 
        #so we divide the values by 128 to restrict values to between -1 and 1.
    except:
        print('error')
X = np.array(X)
Y = np.array(Y)
X = X.reshape(X.shape+(1,)) #dimensions to be the same for X and Y
print(X.shape)
print(Y.shape)


(3, 256, 256, 1)
(3, 256, 256, 2)


In [5]:


#Encoder

model = Sequential()
model.add(Conv2D(64, (3, 3), activation='relu', padding='same', strides=2, input_shape=(256, 256, 1)))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(128, (3,3), activation='relu', padding='same', strides=2))
model.add(Conv2D(256, (3,3), activation='relu', padding='same'))
model.add(Conv2D(256, (3,3), activation='relu', padding='same', strides=2))
model.add(Conv2D(512, (3,3), activation='relu', padding='same'))
model.add(Conv2D(512, (3,3), activation='relu', padding='same'))
model.add(Conv2D(256, (3,3), activation='relu', padding='same'))

#Decoder
#Decoder
#Note: For the last layer we use tanh instead of Relu. 
#This is because we are colorizing the image in this layer using 2 filters, A and B.
#A and B values range between -1 and 1 so tanh (or hyperbolic tangent) is used
#as it also has the range between -1 and 1. 
#Other functions go from 0 to 1.
model.add(Conv2D(128, (3,3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(64, (3,3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(32, (3,3), activation='relu', padding='same'))
model.add(Conv2D(16, (3,3), activation='relu', padding='same'))
model.add(Conv2D(2, (3, 3), activation='tanh', padding='same'))
model.add(UpSampling2D((2, 2)))
model.compile(optimizer='adam', loss='mse' , metrics=['accuracy'])
model.summary()



Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 128, 128, 64)      640       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 128, 128, 128)     73856     
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 64, 64, 128)       147584    
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 64, 64, 256)       295168    
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 32, 32, 256)       590080    
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 32, 32, 512)       1180160   
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 32, 32, 512)       2

In [10]:

model.fit(X,Y,validation_split=0.1, epochs=100, batch_size=16)

model.save('other_files/colorize_autoencoder.model')

###########################################################
#Load saved model and test on images.
#colorize_autoencoder300.model is trained on 300 epochs
#

Epoch 1/100
1/1 [==============================] - 1s 1s/step - loss: 0.0044 - accuracy: 0.8915 - val_loss: 0.0268 - val_accuracy: 0.7451
Epoch 2/100
1/1 [==============================] - 1s 887ms/step - loss: 0.0042 - accuracy: 0.8870 - val_loss: 0.0258 - val_accuracy: 0.7447
Epoch 3/100
1/1 [==============================] - 1s 849ms/step - loss: 0.0041 - accuracy: 0.8905 - val_loss: 0.0215 - val_accuracy: 0.7369
Epoch 4/100
1/1 [==============================] - 1s 806ms/step - loss: 0.0041 - accuracy: 0.8938 - val_loss: 0.0317 - val_accuracy: 0.7436
Epoch 5/100
1/1 [==============================] - 1s 838ms/step - loss: 0.0042 - accuracy: 0.8923 - val_loss: 0.0160 - val_accuracy: 0.7231
Epoch 6/100
1/1 [==============================] - 1s 838ms/step - loss: 0.0047 - accuracy: 0.8909 - val_loss: 0.0310 - val_accuracy: 0.7479
Epoch 7/100
1/1 [==============================] - 1s 860ms/step - loss: 0.0040 - accuracy: 0.8961 - val_loss: 0.0262 - val_accuracy: 0.7470
Epoch 8/100
1/1 

'\ntf.keras.models.load_model(\n    \'other_files/colorize_autoencoder300.model\',\n    custom_objects=None,\n    compile=True)\nimg1_color=[]\nimg1=img_to_array(load_img(\'images/monalisa.png\'))\nimg1 = resize(img1 ,(256,256))\nimg1_color.append(img1)\nimg1_color = np.array(img1_color, dtype=float)\nimg1_color = rgb2lab(1.0/255*img1_color)[:,:,:,0]\nimg1_color = img1_color.reshape(img1_color.shape+(1,))\noutput1 = model.predict(img1_color)\noutput1 = output1*128\nresult = np.zeros((256, 256, 3))\nresult[:,:,0] = img1_color[0][:,:,0]\nresult[:,:,1:] = output1[0]\nimsave("result.png", lab2rgb(result))\n'

In [13]:

tf.keras.models.load_model(
    'other_files/colorize_autoencoder.model',
    custom_objects=None,
    compile=True)
img1_color=[]
img1=img_to_array(load_img('images/some.jpg'))
img1 = resize(img1 ,(256,256))
img1_color.append(img1)
img1_color = np.array(img1_color, dtype=float)
img1_color = rgb2lab(1.0/255*img1_color)[:,:,:,0]
img1_color = img1_color.reshape(img1_color.shape+(1,))
output1 = model.predict(img1_color)
output1 = output1*128
result = np.zeros((256, 256, 3))
result[:,:,0] = img1_color[0][:,:,0]
result[:,:,1:] = output1[0]
imsave("result.png", lab2rgb(result))


Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
